In [1]:
# import requests;                                 # import package used to downlaod data from a web URL
# import json;
import pandas as pd;
# import matplotlib.pyplot as plt;
# import matplotlib;
# import seaborn as sns;
import datetime as dt
import time
import numpy as np
import bisect

# from pandas.io.json import json_normalize

# matplotlib.style.use('ggplot');
# %matplotlib inline

In [27]:
dfHist = pd.read_csv('itemhistory.csv', dtype={'Craft':str, 'DateTime':dt.datetime, 'DefIndex':np.float64, 'Name':str,
                                            'PriceIndex':str, 'QualInt':np.int64, 'Trade':str, 'currency':str,
                                            'timestamp':np.float64, 'value':np.float64, 'value_high':np.float64})
# dfHist = pd.read_csv('itemhistory.csv')
dfHist['DateTime'] = pd.to_datetime(dfHist['DateTime'])
# dfHist.drop_duplicates(inplace=True)
# dfHist = dfHist.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'timestamp'])
dfHist = dfHist.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'timestamp'])
dfHist.sort_index(inplace = True)

In [28]:
#Gets the history values for keys (currency in metal)
dfKey = dfHist.copy(deep = True).loc['Mann Co. Supply Crate Key']

In [29]:
#GEts the history values for metal (currency in USD)
dfMetal = dfHist.copy(deep = True).loc['Refined Metal'].loc[5002].loc[6]

In [30]:
#dates is a sorted list of DateTimes, pivot is a DateTime
#that you want to find the nearest previous date in dates
def prevDate(dates, pivot):
    insertNum = bisect.bisect_left(dates, pivot)
    if (insertNum != len(dates)):
#     if (insertNum != dates.size):
        if (dates[insertNum] == pivot):
            return insertNum
    if(insertNum == 0):
        return 0
    return (insertNum - 1)

In [31]:
#dates is a sorted list of DateTimes, pivot is a DateTime
#that you want to find the nearest previous date in dates
def nextDate(dates, pivot):
    insertNum = bisect.bisect_right(dates, pivot)
    if insertNum > 0:
        if (dates[insertNum - 1] == pivot):
            return insertNum - 1
        if (insertNum == len(dates)):
            return insertNum - 1
    return (insertNum)

In [32]:
dfMetalVal = dfMetal.reset_index().loc[:,'DateTime':]
dfMetalVal.drop('currency', inplace = True, axis = 1)
dfMetalVal['value'] = np.round((dfMetalVal['value'] + dfMetalVal['value_high'])/2, decimals = 2)
# dfMetalVal['value'] = (dfMetalVal['value'] + dfMetalVal['value_high'])/2
dfMetalVal.drop('value_high', inplace = True, axis = 1)

In [33]:
dfKeyVal = dfKey.copy(deep = True)

In [34]:
for ind, ser in dfKeyVal.iterrows():
    dfKeyVal.loc[ind, 'value'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                       dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
    dfKeyVal.loc[ind, 'value_high'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                       dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)

In [35]:
dfKeyVal = dfKeyVal.reset_index().loc[:,'DateTime':]
dfKeyVal.drop('currency', inplace = True, axis = 1)
dfMetalVal['value'] = np.round(dfMetalVal['value'], decimals = 2)
# dfKeyVal['value'] = (dfKeyVal['value'] + dfKeyVal['value_high'])/2
dfKeyVal.drop('value_high', inplace = True, axis = 1)

In [38]:
for ind, ser in dfHist.iterrows():
    if (ser['currency'] == 'metal'):
        dfHist.loc[ind, 'value'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                           dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
        dfHist.loc[ind, 'value_high'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                           dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)
        dfHist.loc[ind, 'currency'] = 'usd'
    elif (ser['currency'] == 'keys'):
        dfHist.loc[ind, 'value'] = np.float64.round(((dfKeyVal.iloc[prevDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'] +
                           dfKeyVal.iloc[nextDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
        dfHist.loc[ind, 'value_high'] = np.float64.round(((dfKeyVal.iloc[prevDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'] +
                           dfKeyVal.iloc[nextDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)
        dfHist.loc[ind, 'currency'] = 'usd'

In [39]:
dfHist.head(30)

DateTime  \
Name                     DefIndex QualInt Trade    Craft     PriceIndex timestamp                        
A Brush with Death       30186    6       Tradable Craftable 0          1381676945 2013-10-13 15:09:05   
                                                                        1383401806 2013-11-02 14:16:46   
                                                                        1383665506 2013-11-05 15:31:46   
                                                                        1383798437 2013-11-07 04:27:17   
                                                                        1383859373 2013-11-07 21:22:53   
                                                                        1384015717 2013-11-09 16:48:37   
                                                                        1384161954 2013-11-11 09:25:54   
                                                                        1384383384 2013-11-13 22:56:24   
                                                                        1384656665 2013-11-17 02:51:05   
                                                                        1398733640 2014-04-29 01:07:20   
                                                                        1398990171 2014-05-02 00:22:51   
                                                                        1405098842 2014-07-11 17:14:02   
                                                                        1408187039 2014-08-16 11:03:59   
                                                                        1409449238 2014-08-31 01:40:38   
                                                                        1414091591 2014-10-23 19:13:11   
                                                                        1414184620 2014-10-24 21:03:40   
                                                                        1416873584 2014-11-24 23:59:44   
                                                                        1417709602 2014-12-04 16:13:22   
                                                                        1418603183 2014-12-15 00:26:23   
                                                                        1418670431 2014-12-15 19:07:11   
                                                                        1418850320 2014-12-17 21:05:20   
                                                                        1428857089 2015-04-12 16:44:49   
                                                                        1430068807 2015-04-26 17:20:07   
                                                                        1430745862 2015-05-04 13:24:22   
                                                                        1432684938 2015-05-27 00:02:18   
                                                                        1436704678 2015-07-12 12:37:58   
                                                                        1440136296 2015-08-21 05:51:36   
                                                                        1452896758 2016-01-15 22:25:58   
A Color Similar to Slate 5052     6       Tradable Craftable 0          1342791340 2012-07-20 13:35:40   
                                                                        1345212999 2012-08-17 14:16:39   

                                                                                   currency  \
Name                     DefIndex QualInt Trade    Craft     PriceIndex timestamp             
A Brush with Death       30186    6       Tradable Craftable 0          1381676945      usd   
                                                                        1383401806      usd   
                                                                        1383665506      usd   
                                                                        1383798437      usd   
                                                                        1383859373      usd   
                                    

In [41]:
dfDrop = dfHist.copy(deep = True)

In [43]:
dfDrop = dfDrop[dfDrop['currency'] == 'usd']

In [46]:
#This date is the first time metal was found. Before that USD Prices are off.
#So we drop everything before that date.
date = pd.to_datetime("2013-01-20 00:49:02")
delta = pd.Timedelta('0 days')

In [48]:
dfDrop = dfDrop[dfDrop['DateTime'] - date >= delta]

In [52]:
dfDrop.to_csv('itemhistoryUSD.csv')